In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
df = pd.read_csv('dataset_train.csv')
train = df[(df['label'] == '2352') | (df['label'] == '1543') | (df['label'] == '2529') | (df['label'] == '2628') | (df['label'] == '1333') | (df['label'] == '2556') | (df['label'] == '1746') | (df['label'] == '2907') | (df['label'] == '2579') | (df['label'] == '3101')]

In [3]:
train = train.reset_index()
train.drop('index',axis=1,inplace=True)

In [4]:
train.head()

,label,path
0,1333,data-train/1333/PM-WWA-20050620-050.jpg
1,1333,data-train/1333/PM-WWA-20050620-153.jpg
2,1333,data-train/1333/PM-WWA-20070526-232.jpg
3,1333,data-train/1333/PM-WWA-20070617-087.jpg
4,1333,data-train/1333/PM-WWA-20070620-023.jpg


In [5]:
train_image = []

for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['path'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)

100%|████████████████████████████████████████████████████████████████████████████████| 207/207 [00:06<00:00, 30.23it/s]


In [6]:
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

(207, 224, 224, 3)

In [7]:
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [8]:
X_train = X_train.reshape(X_train.shape[0], 224, 224, 3)
X_test = X_test.reshape(X_test.shape[0], 224, 224, 3)

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [10]:
X_train /= 255
X_test /= 255

In [11]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [15]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(224, 224, 3), activation = 'relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, (3,3), activation = 'relu')) 
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())

In [16]:
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 128, activation = 'relu')) 
model.add(Dropout(0.2))

In [17]:
model.add(Dense(units = 10, activation = 'softmax')) 
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size = 128, epochs = 60, validation_data = (X_test, y_test))

Train on 165 samples, validate on 42 samples
Epoch 1/60
165/165 [==============================] - ETA: 28s - loss: 2.3028 - accuracy: 0.109 - 118s 715ms/step - loss: 2.4118 - accuracy: 0.0970 - val_loss: 2.3022 - val_accuracy: 0.0714
Epoch 2/60
165/165 [==============================] - ETA: 38s - loss: 2.9535 - accuracy: 0.109 - 168s 1s/step - loss: 2.8083 - accuracy: 0.1212 - val_loss: 2.2998 - val_accuracy: 0.1905
Epoch 3/60
165/165 [==============================] - ETA: 34s - loss: 2.2618 - accuracy: 0.187 - 151s 916ms/step - loss: 2.2581 - accuracy: 0.2121 - val_loss: 2.3018 - val_accuracy: 0.0714
Epoch 4/60
165/165 [==============================] - ETA: 31s - loss: 2.1646 - accuracy: 0.195 - 126s 766ms/step - loss: 2.1391 - accuracy: 0.2182 - val_loss: 2.2980 - val_accuracy: 0.1905
Epoch 5/60
165/165 [==============================] - ETA: 38s - loss: 2.0813 - accuracy: 0.265 - 159s 966ms/step - loss: 2.0423 - accuracy: 0.2667 - val_loss: 2.3015 - val_accuracy: 0.0952
Epoch 6/

Epoch 44/60
165/165 [==============================] - ETA: 28s - loss: 0.0531 - accuracy: 0.984 - 111s 671ms/step - loss: 0.0414 - accuracy: 0.9879 - val_loss: 2.5847 - val_accuracy: 0.1905
Epoch 45/60
165/165 [==============================] - ETA: 24s - loss: 0.0100 - accuracy: 1.000 - 109s 658ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 2.6058 - val_accuracy: 0.1905
Epoch 46/60
165/165 [==============================] - ETA: 26s - loss: 0.0615 - accuracy: 0.984 - 112s 678ms/step - loss: 0.0483 - accuracy: 0.9879 - val_loss: 2.6230 - val_accuracy: 0.1905
Epoch 47/60
165/165 [==============================] - ETA: 26s - loss: 0.0236 - accuracy: 0.992 - 108s 656ms/step - loss: 0.0254 - accuracy: 0.9879 - val_loss: 2.6364 - val_accuracy: 0.1905
Epoch 48/60
165/165 [==============================] - ETA: 27s - loss: 0.0027 - accuracy: 1.000 - 112s 680ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 2.6552 - val_accuracy: 0.1905
Epoch 49/60
165/165 [========================

In [19]:
X_train = base_model.predict(X_train)
X_train.shape

(165, 7, 7, 512)

In [20]:
X_test = base_model.predict(X_test)
X_test.shape

(42, 7, 7, 512)

In [23]:
result = model.evaluate(X_test, y_test)

42/42 [==============================] - ETA:  - 2s 40ms/step


In [24]:
result

[5.312014579772949, 0.095238097012043]

In [9]:
# creating dummies of target variable for train and validation set


In [18]:
base_model = VGG16(weights='imagenet', include_top=False)

In [81]:
X_train = base_model.predict(X_train)
X_train.shape

(165, 7, 7, 512)

In [82]:
X_test = base_model.predict(X_test)
X_test.shape

(42, 7, 7, 512)

In [12]:
X_test.shape

(42, 224, 224, 3)

ValueError: cannot reshape array of size 24837120 into shape (165,25088)

In [84]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [85]:
X_train.shape

(165, 25088)

In [86]:
model = Sequential()

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(101, activation='softmax'))

In [87]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='max')

In [88]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [110]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

ValueError: Error when checking input: expected sequential_6_input to have 2 dimensions, but got array with shape (165, 25088, 2)

In [114]:
y_train

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]]], dtype=float32)

In [98]:
X_train

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]], dtype=float32)

In [109]:
from keras.utils import to_categorical

X_test =  to_categorical(X_test)

In [94]:
y_train = y_train.astype('float32')
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [61]:
X_train.shape[0], 28,28,1

y_test = X_test.astype('float32')

In [ ]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)